In [1]:
# Libraries needed for NLP
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

# Libraries needed for Tensorflow processing
import tensorflow as tf
import numpy as np
import random
import json

[nltk_data] Downloading package punkt to C:\Users\Ankit
[nltk_data]     Choubey\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [24]:
with open('C:/Users/Ankit Choubey/Downloads/intents.json','r') as f:
    intents = json.load(f)

In [3]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'chatbot',
   'patterns': ['Who built this chatbot?',
    'who is your creator?',
    'who is the creator of this chatbot?',
    'Tell me about Chatbot',
    'What is this chatbot name?'],
   'responses': ['Hi, I am Bud your chat buddy and I am designed by Ankit.',
    'Thanks for asking. I am designed by Ankit Choubey.',
    'I am a Bud your chatbot.']},
  {'tag': 'location',
   'patterns': ['What is your locat

In [4]:
words = []
classes = []
documents = []
ignore = ['?']
# loop through each sentence in the intent's patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each and every word in the sentence
        w = nltk.word_tokenize(pattern)
        # add word to the words list
        words.extend(w)
        # add word(s) to documents
        documents.append((w, intent['tag']))
        # add tags to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [5]:
# Perform stemming and lower each word as well as remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

# remove duplicate classes
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

31 documents
8 classes ['about', 'chatbot', 'connect', 'goodbye', 'greeting', 'location', 'movies', 'thanks']
55 unique stemmed words ["'s", 'about', 'account', 'address', 'ani', 'anyon', 'are', 'built', 'bye', 'can', 'chatbot', 'connect', 'creator', 'day', 'favourit', 'give', 'good', 'goodby', 'hello', 'help', 'hi', 'how', 'i', 'is', 'later', 'link', 'locat', 'me', 'media', 'movi', 'name', 'of', 'out', 'reach', 'recommend', 'see', 'social', 'some', 'suggest', 'tell', 'thank', 'that', 'the', 'there', 'thi', 'to', 'way', 'we', 'what', 'where', 'which', 'who', 'you', 'your', 'yourself']


In [6]:
# create training data
training = []
output = []
# create an empty array for output
output_empty = [0] * len(classes)

# create training set, bag of words for each sentence
for doc in documents:
    # initialize bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stemming each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is '1' for current tag and '0' for rest of other tags
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffling features and turning it into np.array
random.shuffle(training)
training = np.array(training,dtype = list)

# creating training lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [7]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10,input_shape=(len(train_x[0]),)))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Dense(len(train_y[0]), activation='softmax'))
model.compile(tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy']) 

In [8]:
model.fit(np.array(train_x), np.array(train_y), epochs=500, batch_size=8, verbose=1)
model.save("model.sav")

Epoch 1/500
4/4 [==============================] - 5s 16ms/step - loss: 2.1738 - accuracy: 0.0323
Epoch 2/500
4/4 [==============================] - 0s 12ms/step - loss: 2.1539 - accuracy: 0.0323
Epoch 3/500
4/4 [==============================] - 0s 6ms/step - loss: 2.1361 - accuracy: 0.0645
Epoch 4/500
4/4 [==============================] - 0s 12ms/step - loss: 2.1189 - accuracy: 0.0645
Epoch 5/500
4/4 [==============================] - 0s 10ms/step - loss: 2.1030 - accuracy: 0.0645
Epoch 6/500
4/4 [==============================] - 0s 12ms/step - loss: 2.0872 - accuracy: 0.0645
Epoch 7/500
4/4 [==============================] - 0s 7ms/step - loss: 2.0712 - accuracy: 0.0645
Epoch 8/500
4/4 [==============================] - 0s 8ms/step - loss: 2.0562 - accuracy: 0.0968
Epoch 9/500
4/4 [==============================] - 0s 9ms/step - loss: 2.0415 - accuracy: 0.0968
Epoch 10/500
4/4 [==============================] - 0s 9ms/step - loss: 2.0263 - accuracy: 0.0968
Epoch 11/500
4/4 [======

INFO:tensorflow:Assets written to: model.sav\assets


In [9]:
import pickle
pickle.dump( {'words':words, 'classes':classes}, open( "training_data", "wb" ) )

In [10]:
from keras.models import load_model
model = load_model("model.sav")

In [11]:
# restoring all the data structures
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']

In [12]:
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [13]:
def clean_up_sentence(sentence):
    # tokenizing the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stemming each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# returning bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words):
    # tokenizing the pattern
    sentence_words = clean_up_sentence(sentence)
    # generating bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
    bag=np.array(bag)
    return(bag)

In [14]:
ERROR_THRESHOLD = 0.30
def classify(sentence):
    # generate probabilities from the model
    bag = bow(sentence, words)
    results = model.predict(np.array([bag]))
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results[0]) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # a random response from the intent
                    return print(random.choice(i['responses']))

            results.pop(0)

In [15]:
response('Where are you located?')

1/1 [==============================] - 0s 291ms/step
You can visit India to meet us


In [16]:
response('That is helpful')

1/1 [==============================] - 0s 38ms/step
My pleasure


In [17]:
response('Bye')

1/1 [==============================] - 0s 46ms/step
Bye! Come back again soon.


In [18]:
response("Who are you?")

1/1 [==============================] - 0s 153ms/step
Hi, I am Bud, I was brought to life by Ankit Choubey.


In [19]:
response("who is your creator?")

1/1 [==============================] - 0s 92ms/step
Thanks for asking. I am designed by Ankit Choubey.


In [26]:

response("where can we connect")

1/1 [==============================] - 0s 165ms/step
You can connect me on Linkedin 
 Linkedin - https://www.linkedin.com/in/ankit-choubey-192b28251/


In [32]:
response("suggest me a movie")

1/1 [==============================] - 0s 50ms/step
3 idiots
